#### [bosch kaggle info](https://www.kaggle.com/c/bosch-production-line-performance/data)

[pyspark doc](https://spark.apache.org/docs/2.0.0-preview/api/python/_modules/pyspark/sql/session.html)

** file path **
/home/h/fun/bosch/raw_data

Data Files

- ** 目标是要预测哪个部分会在质量控制过程中失败，用'Response' = 1表示该状态。**
- ** 数据集包含了非常大的匿名特征数据。**
- ** 数据已经按照类型切成两类，数值型和类别型两个大文件。日期特征给每个测量值提供了发生的时间戳。每个日期列都对应于每个特征的生成日期，如L0_S0_D1是特征列L0_S0_F0的日期列。 ** 
- ** 难点： 量大，正负样本的极度不均衡 **

File descriptions

* train_numeric.csv - the training set numeric features (this file contains the 'Response' variable) 
* test_numeric.csv - the test set numeric features (you must predict the 'Response' for these Ids)  
* train_categorical.csv - the training set categorical features  
* test_categorical.csv - the test set categorical features  
* train_date.csv - the training set date features  
* test_date.csv - the test set date features  
* sample_submission.csv - a sample submission file in the correct format 

In [15]:
%cd raw_data/
%ls
%cd ..

/home/h/kaggle_fun/bosch/raw_data
down_data/            test_numeric.csv       train_numeric.csv
test_categorical.csv  train_categorical.csv  zips/
test_date.csv         train_date.csv
/home/h/kaggle_fun/bosch


In [16]:
import pandas as pd
import os

In [17]:
train_date = pd.read_csv("raw_data/train_date.csv", nrows=100)
train_category = pd.read_csv("raw_data/train_categorical.csv", nrows=100)
train_numer = pd.read_csv("raw_data/train_numeric.csv", nrows=1000)

In [18]:
train_category.head()

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
train_date.head()

,Id,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,...,L3_S50_D4246,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263
0,4,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
train_numer.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [22]:
train_category.describe

<bound method DataFrame.describe of      Id  L0_S1_F25  L0_S1_F27  L0_S1_F29  L0_S1_F31  L0_S2_F33  L0_S2_F35  \
0     4        NaN        NaN        NaN        NaN        NaN        NaN   
1     6        NaN        NaN        NaN        NaN        NaN        NaN   
2     7        NaN        NaN        NaN        NaN        NaN        NaN   
3     9        NaN        NaN        NaN        NaN        NaN        NaN   
4    11        NaN        NaN        NaN        NaN        NaN        NaN   
5    13        NaN        NaN        NaN        NaN        NaN        NaN   
6    14        NaN        NaN        NaN        NaN        NaN        NaN   
7    16        NaN        NaN        NaN        NaN        NaN        NaN   
8    18        NaN        NaN        NaN        NaN        NaN        NaN   
9    23        NaN        NaN        NaN        NaN        NaN        NaN   
10   26        NaN        NaN        NaN        NaN        NaN        NaN   
11   27        NaN        NaN        NaN

In [23]:
from operator import add

counts = sc.parallelize([1,2,3,4]).map(lambda x: x^2).reduce(add)
print type(counts)
print counts
# for (word, count) in output:
#         print("%s: %i" % (word, count))

<type 'int'>
10


In [24]:
cat_data =sc.textFile("/home/h/kaggle_fun/bosch/raw_data/train_categorical.csv")

In [33]:
cat_data.count()

1183748

In [34]:
spark = SparkSession.builder \
    .master("spark://h:7077") \
    .appName("Word Count") \
    .getOrCreate()

In [35]:
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"
train_cat = "train_categorical.csv"
train_numeric = "train_numeric.csv"
train_date= "train_date.csv"
train_cat_df = spark.read.csv(file_dir + train_cat, header=True, inferSchema=True)
train_numerer_df = spark.read.csv(file_dir + train_numeric, header=True, inferSchema=True)
train_date_df = spark.read.csv(file_dir + train_date, header=True, inferSchema=True)

In [28]:
train_numerer_df.registerTempTable("train_num")

In [29]:
res = spark.sql("select Response, sum(Response) from train_num group by Response")

In [37]:
print res.collect()

[Row(Response=1, sum(Response)=6879), Row(Response=0, sum(Response)=0)]


In [38]:
print spark.sql("select count(*) from train_num ").collect()

[Row(count(1)=1183747)]


In [39]:
print 1183747-6879

1176868


In [59]:
train_sample = spark.sql("select *, rand() from train_num where Response = 0 or (Response = 1 and rand() < 1.2*6879/1183747)")

In [61]:
train_sample.head(3)

[Row(Id=4, L0_S0_F0=0.03, L0_S0_F2=-0.034, L0_S0_F4=-0.197, L0_S0_F6=-0.179, L0_S0_F8=0.118, L0_S0_F10=0.116, L0_S0_F12=-0.015, L0_S0_F14=-0.032, L0_S0_F16=0.02, L0_S0_F18=0.083, L0_S0_F20=-0.273, L0_S0_F22=-0.273, L0_S1_F24=-0.271, L0_S1_F28=0.167, L0_S2_F32=-0.213, L0_S2_F36=-0.023, L0_S2_F40=-0.192, L0_S2_F44=-0.088, L0_S2_F48=0.001, L0_S2_F52=0, L0_S2_F56=0.01, L0_S2_F60=-0.223, L0_S2_F64=-0.03, L0_S3_F68=None, L0_S3_F72=None, L0_S3_F76=None, L0_S3_F80=None, L0_S3_F84=None, L0_S3_F88=None, L0_S3_F92=None, L0_S3_F96=None, L0_S3_F100=None, L0_S4_F104=-0.001, L0_S4_F109=-0.004, L0_S5_F114=None, L0_S5_F116=None, L0_S6_F118=None, L0_S6_F122=None, L0_S6_F132=None, L0_S7_F136=-0.164, L0_S7_F138=-0.077, L0_S7_F142=0.06, L0_S8_F144=-0.157, L0_S8_F146=0.0, L0_S8_F149=0.001, L0_S9_F155=None, L0_S9_F160=None, L0_S9_F165=None, L0_S9_F170=None, L0_S9_F175=None, L0_S9_F180=None, L0_S9_F185=None, L0_S9_F190=None, L0_S9_F195=None, L0_S9_F200=None, L0_S9_F205=None, L0_S9_F210=None, L0_S10_F219=None,

In [58]:
print 6879/1183747.0

0.00581120796927
